# STAT 301 Final Group Project

In [10]:
library(dplyr)
library(tidymodels)
library(infer)
library(cowplot)
library(GGally)
library(ggplot2)
library(grid)
library(gridExtra) 

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use tidymodels_prefer() to resolve common conflicts.


Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp


Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    co

## Importing the Dataset

In [7]:
# Load data with correct columns
census_data <- read_csv("https://github.com/whalebeavercat/STAT-301-Census-Project/blob/main/data/adult.data?raw=TRUE", col_names = FALSE, show_col_types = FALSE)
col_names = c("age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income")
colnames(census_data) <- col_names

head(census_data)
nrow(census_data)

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


[1] 32561

`Table 1`: Initial Dataset

### Dataset Summary

The dataset being explored is the [1994 Census Income Data Set](https://archive.ics.uci.edu/dataset/2/adult) that explores personal annual income of US citizens in 1994. It is a reasonably clean dataset but it has some missing values in the coloumns *workclass*, *occupation*, and *native-country*. These missing values have been entered as "?" and will be removed from the dataset. The variables present in the dataset are:

- `age`: numeric variable, the age of a person in years. Only individuals with `age` > 16 were considered.
- `workclass`: categorical variable, type chr, represents the type of employement, such as Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- `fnlwgt`: numeric variable, represent the final weight, meaning the number of people the census believes are represented by all the features in the row
- `education`: categorical variable, type chr, represents the highest education level obtained, such as Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- `education-num`: numeric variable, the highest education level obtained in numeric form
- `marital-status`: categorical variable, type chr, represents the marital status for both civilian and Armed Forces marriages, such as Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- `occupation`: categorical variable, type chr, represents the type of occupation the person has, such Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- `relationship`: categorical variable, type chr, represents the relations the person has to others, similar to `maritial-status`, such as Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- `race`: categorical variable, type chr, represents the race of the individual, such as White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- `sex`: categorical variable, type chr, represents the biological sex, such as Female, Male.
- `capital-gain`: numeric variable, the capital gain for the individual or profit made by selling assets.
- `capital-loss`: numeric variable, the capital loss for the individual or loss from selling assets.
- `hours-per-week`: numeric variable, the number of hours per week the individual works.
- `native-country`: categorical variable, the country where the individual is born, such as United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- `income`: categorical variable, type logical, whether the individual makes <=50K or > 50K

### Question
The proposed question for this project is: Can we predict whether an individual earns >50K USD per year based on an individuals' `age`, `race`, education level (`education-num`), and `sex`. The aim of this project is to explore any significant bias in the income of an individual and thier personal background, which can be used to further explore a broader interest in understanding the complex interplay of numerous demographic factors in shaping individuals' income.



## Exploratory Data Analysis

### Cleaning and Wrangling the Dataset
Any row which contains the value "?" would be filtered out and the filtered dataset would then be processed again to remove all the unnecessary coloumns from the filtered dataset. This leaves us with a dataset with 30162 rows and 5 columns

In [9]:
census_data_clean <- census_data %>%
    filter_all(all_vars(. != "?")) %>%
    select(age, `education-num`, race, sex, income)
head(census_data_clean)
nrow(census_data_clean)

age,education-num,race,sex,income
<dbl>,<dbl>,<chr>,<chr>,<chr>
39,13,White,Male,<=50K
50,13,White,Male,<=50K
38,9,White,Male,<=50K
53,7,Black,Male,<=50K
28,13,Black,Female,<=50K
37,14,White,Female,<=50K


[1] 30162

`Table 2`: Wrangled and Cleaned Dataset

Given that the origninal dataset is a census dataset with large populations and various categories, it is more appropriate to take a sample from it. Taking a random sample can help identify trends, outliers, and potential issues without processing the entire dataset, improving computational efficiency. A simple random sample of size 1000 should be a good start as the sample is small enough to avoid introducing any biases. This sample dataset will then be s

In [14]:
sample_data <- census_data_clean %>%
    sample_n(size = 1000)
head(sample_data)

df_split <- initial_split(sample_data, prop = 0.8, strata = age)
df_train <- training(df_split)
df_test <- testing(df_split)

age,education-num,race,sex,income
<dbl>,<dbl>,<chr>,<chr>,<chr>
37,13,White,Female,<=50K
24,9,White,Male,<=50K
17,6,White,Male,<=50K
23,10,White,Female,<=50K
19,10,White,Female,<=50K
44,9,White,Male,<=50K


`Table 3`: Sampled Data

### Data Visualization